# Phase 2: Open System Dynamics and Decoherence

This notebook demonstrates the Lindblad master equation solver for open quantum systems, including T1 (energy relaxation) and T2 (dephasing) decoherence.

**Author:** Orchestrator Agent  
**Date:** 2025-01-27  
**SOW Reference:** Phase 2.3 - Open System Dynamics  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import sys
sys.path.insert(0, '..')

from src.hamiltonian.lindblad import LindbladEvolution, DecoherenceParams, thermal_state
from src.hamiltonian.drift import DriftHamiltonian

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Decoherence Parameters

Real quantum systems are subject to decoherence from environmental coupling:

- **T1 (Energy Relaxation):** Spontaneous emission rate, |1⟩ → |0⟩
- **T2 (Total Dephasing):** Phase randomization rate
- **Constraint:** T2 ≤ 2×T1 (fundamental quantum limit)

Typical values for superconducting qubits:
- T1: 10-100 μs
- T2: 1-100 μs

In [ ]:
# Define decoherence parameters (times in microseconds)
T1 = 50.0  # Energy relaxation time
T2 = 30.0  # Dephasing time (< 2*T1)

decoherence = DecoherenceParams(T1=T1, T2=T2, temperature=0.0)

print(f"Decoherence Parameters:")
print(f"  T1 (energy relaxation): {decoherence.T1:.1f} μs")
print(f"  T2 (total dephasing):   {decoherence.T2:.1f} μs")
print(f"  T2 / (2*T1):            {decoherence.T2 / (2*decoherence.T1):.3f}")
print(f"\nPure dephasing rate: γ_φ = 1/T2 - 1/(2*T1) = {1/T2 - 1/(2*T1):.6f} μs⁻¹")

## 2. T1 Relaxation: Energy Decay

T1 characterizes how quickly an excited qubit decays to the ground state. The population decays exponentially:

$$P_1(t) = e^{-t/T_1}$$

In [ ]:
# Create Lindblad solver
H = 0.5 * 5.0 * qt.sigmaz()  # Drift Hamiltonian (5 MHz)
lindblad = LindbladEvolution(H, decoherence)

# Simulate T1 relaxation from excited state
times, populations = lindblad.relaxation_curve(
    initial_state='excited',
    max_time=5 * T1
)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(times, populations, 'b-', linewidth=2, label='Simulated')
ax.plot(times, np.exp(-times / T1), 'r--', linewidth=2, label=f'Ideal: exp(-t/{T1})')
ax.axhline(1/np.e, color='gray', linestyle=':', alpha=0.7, label='1/e ≈ 0.368')
ax.axvline(T1, color='gray', linestyle=':', alpha=0.7)

ax.set_xlabel('Time (μs)', fontsize=14)
ax.set_ylabel('Excited State Population P₁(t)', fontsize=14)
ax.set_title(f'T1 Relaxation Curve (T1 = {T1} μs)', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Verify decay at t=T1
idx_t1 = np.argmin(np.abs(times - T1))
print(f"\nPopulation at t = T1: {populations[idx_t1]:.4f} (expected: {1/np.e:.4f})")
print(f"Relative error: {np.abs(populations[idx_t1] - 1/np.e) / (1/np.e) * 100:.2f}%")

## 3. T2 Dephasing: Ramsey Experiment

A Ramsey experiment measures T2 by:
1. Prepare superposition state (π/2 pulse)
2. Free evolution with detuning
3. Measure oscillating signal with decay envelope ∝ exp(-t/T2)

In [ ]:
# Simulate Ramsey experiment
detuning = 0.5  # MHz (frequency offset)
times_ramsey, signal = lindblad.ramsey_experiment(
    detuning=detuning,
    max_time=3 * T2
)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Full signal
ax1.plot(times_ramsey, signal, 'b-', linewidth=1.5, label='Ramsey Signal')
envelope = np.exp(-times_ramsey / T2)
ax1.plot(times_ramsey, envelope, 'r--', linewidth=2, label=f'Decay Envelope: exp(-t/{T2})')
ax1.plot(times_ramsey, -envelope, 'r--', linewidth=2)
ax1.axvline(T2, color='gray', linestyle=':', alpha=0.7, label=f't = T2 = {T2} μs')

ax1.set_xlabel('Free Evolution Time (μs)', fontsize=14)
ax1.set_ylabel('⟨σ_x⟩', fontsize=14)
ax1.set_title(f'Ramsey Experiment: T2 Dephasing (detuning = {detuning} MHz)', fontsize=16)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# Zoom in on first few oscillations
zoom_time = min(50, T2)
zoom_mask = times_ramsey <= zoom_time
ax2.plot(times_ramsey[zoom_mask], signal[zoom_mask], 'b-', linewidth=2)
ax2.set_xlabel('Time (μs)', fontsize=14)
ax2.set_ylabel('⟨σ_x⟩', fontsize=14)
ax2.set_title('Zoom: First Few Oscillations', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nRamsey fringe period: {2*np.pi / detuning:.2f} μs")
print(f"Decay constant: T2 = {T2:.1f} μs")

## 4. Comparison: Open vs. Closed System

Compare evolution with and without decoherence to see the impact on quantum state fidelity and purity.

In [ ]:
# Initial state: superposition
psi_plus = (qt.basis(2, 0) + qt.basis(2, 1)).unit()
rho0 = psi_plus * psi_plus.dag()

# Time evolution
times_comp = np.linspace(0, 100, 500)

# Compare with unitary evolution
comparison = lindblad.compare_with_unitary(rho0, times_comp)

# Plot results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Fidelity
ax1.plot(times_comp, comparison['fidelities'], 'b-', linewidth=2)
ax1.axhline(0.99, color='r', linestyle='--', alpha=0.7, label='99% fidelity threshold')
ax1.set_xlabel('Time (μs)', fontsize=14)
ax1.set_ylabel('Fidelity with Ideal Evolution', fontsize=14)
ax1.set_title('Decoherence Impact: Fidelity Loss', fontsize=16)
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1.05])

# Purity
ax2.plot(times_comp, comparison['purity'], 'g-', linewidth=2)
ax2.axhline(1.0, color='r', linestyle='--', alpha=0.7, label='Pure state')
ax2.axhline(0.5, color='orange', linestyle='--', alpha=0.7, label='Maximally mixed')
ax2.set_xlabel('Time (μs)', fontsize=14)
ax2.set_ylabel('Purity Tr(ρ²)', fontsize=14)
ax2.set_title('State Purity: Pure → Mixed', fontsize=16)
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1.05])

plt.tight_layout()
plt.show()

# Statistics
print(f"\nFidelity Statistics:")
print(f"  Initial:    {comparison['fidelities'][0]:.6f}")
print(f"  Final:      {comparison['fidelities'][-1]:.6f}")
print(f"  Loss:       {(1 - comparison['fidelities'][-1]) * 100:.2f}%")

print(f"\nPurity Statistics:")
print(f"  Initial:    {comparison['purity'][0]:.6f}")
print(f"  Final:      {comparison['purity'][-1]:.6f}")

## 5. Gate Fidelity with Decoherence

How does gate fidelity degrade due to decoherence? We'll simulate an X-gate (π pulse) at different speeds.

In [ ]:
# Test X-gate at various speeds
gate_times = np.linspace(5, 100, 20)  # μs
fidelities = []

U_target = qt.sigmax()  # X-gate
rho0_gate = qt.basis(2, 0) * qt.basis(2, 0).dag()

for gate_time in gate_times:
    # Create Hamiltonian for X-gate: constant drive
    omega = np.pi / gate_time  # Rabi frequency for π pulse
    H_gate = [0 * qt.qeye(2), [qt.sigmax(), lambda t, args: omega]]
    
    # Create Lindblad solver
    lindblad_gate = LindbladEvolution(H_gate, decoherence)
    
    # Compute fidelity
    fid = lindblad_gate.gate_fidelity_with_decoherence(
        U_target, rho0_gate, gate_time, n_steps=500
    )
    fidelities.append(fid)

fidelities = np.array(fidelities)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(gate_times, fidelities, 'bo-', linewidth=2, markersize=8, label='Gate Fidelity')
ax.axhline(0.99, color='r', linestyle='--', alpha=0.7, label='99% threshold')
ax.axhline(0.999, color='orange', linestyle='--', alpha=0.7, label='99.9% threshold')
ax.axvline(T1, color='gray', linestyle=':', alpha=0.7, label=f'T1 = {T1} μs')
ax.axvline(T2, color='purple', linestyle=':', alpha=0.7, label=f'T2 = {T2} μs')

ax.set_xlabel('Gate Time (μs)', fontsize=14)
ax.set_ylabel('Gate Fidelity', fontsize=14)
ax.set_title('X-Gate Fidelity vs. Gate Speed (with T1/T2 Decoherence)', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_ylim([min(fidelities) * 0.95, 1.01])

plt.tight_layout()
plt.show()

# Find optimal gate time
idx_best = np.argmax(fidelities)
print(f"\nBest Gate Time: {gate_times[idx_best]:.1f} μs")
print(f"Best Fidelity:  {fidelities[idx_best]:.6f}")
print(f"\nFastest 99% gate:   {gate_times[fidelities >= 0.99][0] if any(fidelities >= 0.99) else 'N/A':.1f} μs")
print(f"Fastest 99.9% gate: {gate_times[fidelities >= 0.999][0] if any(fidelities >= 0.999) else 'N/A':.1f} μs")

## 6. Bloch Sphere Visualization

Visualize how decoherence affects trajectories on the Bloch sphere.

In [ ]:
from src.hamiltonian.evolution import bloch_trajectory

# Initial state on equator
psi0_bloch = (qt.basis(2, 0) + qt.basis(2, 1)).unit()
rho0_bloch = psi0_bloch * psi0_bloch.dag()

# Evolve with small drift
H_bloch = 0.5 * 0.2 * qt.sigmaz()  # Slow precession
lindblad_bloch = LindbladEvolution(H_bloch, decoherence)

times_bloch = np.linspace(0, 150, 300)
result_bloch = lindblad_bloch.evolve(rho0_bloch, times_bloch)

# Compute Bloch coordinates
bloch_coords = []
for rho in result_bloch.states:
    x = qt.expect(qt.sigmax(), rho)
    y = qt.expect(qt.sigmay(), rho)
    z = qt.expect(qt.sigmaz(), rho)
    bloch_coords.append([x, y, z])

bloch_coords = np.array(bloch_coords)

# Plot trajectory
fig = plt.figure(figsize=(14, 6))

# 3D trajectory
ax1 = fig.add_subplot(121, projection='3d')
colors = plt.cm.viridis(np.linspace(0, 1, len(times_bloch)))
ax1.plot(bloch_coords[:, 0], bloch_coords[:, 1], bloch_coords[:, 2], 
         'b-', linewidth=2, alpha=0.7)
ax1.scatter(bloch_coords[0, 0], bloch_coords[0, 1], bloch_coords[0, 2], 
            color='green', s=100, marker='o', label='Start')
ax1.scatter(bloch_coords[-1, 0], bloch_coords[-1, 1], bloch_coords[-1, 2], 
            color='red', s=100, marker='x', label='End')

# Draw Bloch sphere
u, v = np.mgrid[0:2*np.pi:30j, 0:np.pi:20j]
x_sphere = np.cos(u)*np.sin(v)
y_sphere = np.sin(u)*np.sin(v)
z_sphere = np.cos(v)
ax1.plot_wireframe(x_sphere, y_sphere, z_sphere, color='gray', alpha=0.1)

ax1.set_xlabel('⟨σ_x⟩', fontsize=12)
ax1.set_ylabel('⟨σ_y⟩', fontsize=12)
ax1.set_zlabel('⟨σ_z⟩', fontsize=12)
ax1.set_title('Bloch Sphere Trajectory\n(with Decoherence)', fontsize=14)
ax1.legend()

# Radius vs time (purity decay)
ax2 = fig.add_subplot(122)
radius = np.linalg.norm(bloch_coords, axis=1)
ax2.plot(times_bloch, radius, 'b-', linewidth=2, label='Bloch Vector Length')
ax2.axhline(1.0, color='r', linestyle='--', alpha=0.7, label='Pure State (r=1)')
ax2.set_xlabel('Time (μs)', fontsize=14)
ax2.set_ylabel('Bloch Vector Radius', fontsize=14)
ax2.set_title('Purity Decay: r(t) = √(⟨σ_x⟩² + ⟨σ_y⟩² + ⟨σ_z⟩²)', fontsize=14)
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBloch Vector Statistics:")
print(f"  Initial radius: {radius[0]:.6f}")
print(f"  Final radius:   {radius[-1]:.6f}")
print(f"  Shrinkage:      {(1 - radius[-1]) * 100:.2f}%")

## 7. Key Takeaways

### Physics Insights
1. **T1 Relaxation:** Excited states decay exponentially with rate γ₁ = 1/T1
2. **T2 Dephasing:** Coherent superpositions lose phase information at rate γ₂ = 1/T2
3. **Fundamental Limit:** T2 ≤ 2×T1 (T2 includes T1 contributions)
4. **Mixed States:** Decoherence transforms pure states → mixed states (purity < 1)

### Gate Design Implications
1. **Speed vs. Fidelity:** Faster gates minimize decoherence but risk control errors
2. **Optimal Gate Time:** Trade-off between decoherence (favors fast) and pulse bandwidth (favors slow)
3. **Typical Sweet Spot:** T_gate ≈ T2/5 to T2/3 for high fidelity

### Experimental Validation
1. **T1 Measurement:** Prepare |1⟩, wait variable time, measure population
2. **T2 Measurement:** Ramsey or spin-echo experiments
3. **Gate Benchmarking:** Randomized benchmarking, process tomography

---

**Next Steps:** Use optimal control (GRAPE/Krotov) to design pulses that maximize fidelity under realistic T1/T2 constraints!

## Appendix: Mathematical Framework

### Lindblad Master Equation
$$\frac{d\rho}{dt} = -i[H, \rho] + \sum_k \left( L_k \rho L_k^\dagger - \frac{1}{2}\{L_k^\dagger L_k, \rho\} \right)$$

### Collapse Operators
- **T1:** $L_1 = \sqrt{\gamma_1} \sigma_- = \sqrt{1/T_1} |0\rangle\langle 1|$
- **Pure Dephasing:** $L_\phi = \sqrt{\gamma_\phi} \sigma_z$ where $\gamma_\phi = 1/T_2 - 1/(2T_1)$

### Physical Constraints
1. **Trace Preservation:** $\text{Tr}(\rho) = 1$ for all $t$
2. **Positivity:** $\rho \geq 0$ (valid density matrix)
3. **T2 Limit:** $T_2 \leq 2T_1$ (ensures $\gamma_\phi \geq 0$)
4. **Purity Decrease:** $\text{Tr}(\rho^2)$ non-increasing (entropy grows)